In [1]:
import sys
sys.path.insert(0, '../qdmt/')

In [2]:
%matplotlib notebook

In [3]:
import vumps as v
from vumpt_tools import mixedCanonical
from hamiltonian import TransverseIsing
import optimise_state as os

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from ncon import ncon

In [5]:
# Load example Hamiltonian
h = TransverseIsing(1, 0.2, 2)
h = v.tensorOperator(h)

In [6]:
# Load example density matrix
D = 2
d = 2
A = v.createMPS(D, d)
A = v.normalizeMPS(A)

In [7]:
I = np.eye(4).reshape(2, 2, 2, 2)

In [8]:
def uniform_to_two_site(A):
    TM = ncon([A, A.conj()], ((-1, 1, -3), (-2, 1, -4))).reshape(D**2, D**2)

    vals, vecs = np.linalg.eig(TM)
    r = vecs[:, 0]
    #eval_r = vals[0]
    vals, vecs = np.linalg.eig(TM.T)
    l = vecs[:, 0]
    #eval_l = vals[0]
    
    #lTM = l @ TM
    #print(np.allclose(lTM, eval_l * l))
    
    #TMr = TM @ r
    #print(np.allclose(TMr, eval_r*r))
    
    rho = ncon([l.reshape(2, 2), A, A, A.conj(), A.conj(), r.reshape(2, 2)], 
              ((1, 2), (1, -1, 4), (4, -2, 6), (2, -3, 7), (7, -4, 8), (6, 8)))
    return rho

In [9]:
def trace_dist(A, B):
    assert A.shape == B.shape
    assert len(A.shape) == 2

    dist = A - B
    return np.real(np.trace(dist @ dist.conj().T))


In [10]:
rho0 = uniform_to_two_site(A)

In [11]:
Al, Ac, Ar, C = mixedCanonical(A)
rho0_mixed = os.mixed_state_to_two_site_rho(Al, C, Ar)

In [12]:
np.save('rho_uniform.npy', rho0)
np.save('rho_mixed.npy', rho0_mixed)

In [12]:
print(trace_dist(rho0.reshape(4, 4), rho0_mixed.reshape(4, 4)))

0.0005025800952197651


In [13]:
rho = I - rho0
#rho = rho / np.linalg.norm(rho)

### hermitian

First verify both matrices are Hermitian

In [14]:
print(rho.shape)

(2, 2, 2, 2)


In [15]:
print(h.shape)

(2, 2, 2, 2)


In [16]:
h_dagger = h.conj().transpose(2, 3, 0, 1)

In [17]:
np.allclose(h, h_dagger)

True

In [18]:
rho_dagger = rho.conj().transpose(2, 3, 0, 1)

In [19]:
np.allclose(rho, rho_dagger)

True

### norm

In [20]:
print(np.linalg.norm(h))

2.0199009876724157


In [21]:
print(np.linalg.norm(rho))

1.7273099206023437


### Applying VUMPS

In [22]:
from scipy.sparse.linalg import bicgstab, LinearOperator, gmres
from scipy.sparse.linalg import eigs

In [61]:
def sumLeft(AL, C, h, tol=1e-8):
    tol = max(tol, 1e-14)
    D, d, _ = AL.shape
    m = len(h.shape) // 2

    h0_edge = list(range(1, 2*m+1))
    curr_i = 2*m+1
    edges_A = [[curr_i, m+1, curr_i + 1]]
    edges_A_dag = [[curr_i, 1, curr_i + 2]]
    curr_i = curr_i + 1

    for i in range(1, m):
        edges_A.append([curr_i, m+1+i, curr_i+2])
        edges_A_dag.append([curr_i+1, i+1, curr_i+3])
        curr_i += 2

    edges_A[-1][-1] = -1
    edges_A_dag[-1][-1] = -2

    edges = [*edges_A, h0_edge, *edges_A_dag]
    tensors = [*[AL] * m, h, *[AL.conj()]*m]

    Hl = ncon(tensors, edges)
    # To be used in Ax = b solver
    b = Hl.reshape(D**2).conj()

    l = np.eye(D).reshape(-1)
    r = (C @ C.conj().T).reshape(-1)
    Et = v.Etilde(AL, l, r)

    A_ = Et.conj().T # So that Ax = b
    mvec = lambda v: A_ @ v
    A = LinearOperator((D**2, D**2), matvec=mvec)


    Lh, ecode = v.gmres(A, b, tol=tol)

    if ecode != 0:
        print('Sum Left gmres ecode: ', ecode)
        #raise Exception("Sum Left did not converge")
    return Lh

def sumRight(AR, C, h, tol=1e-8):
    D, d, _ = AR.shape
    m = len(h.shape) // 2

    h0_edge = list(range(1, 2*m+1))
    curr_i = 2*m+1
    curr_i += 1
    edges_A = [[curr_i, m+1, curr_i + 2]]
    edges_A_dag = [[curr_i+1, 1, curr_i + 3]]
    curr_i = curr_i + 2

    for i in range(1, m):
        edges_A.append([curr_i, m+1+i, curr_i+2])
        edges_A_dag.append([curr_i+1, i+1, curr_i+3])
        curr_i += 2

    edges_A_dag[-1][-1] = curr_i
    edges_A[0][0] = -1
    edges_A_dag[0][0] = -2

    edges = edges_A + [h0_edge] + edges_A_dag
    tensors = [*[AR]*m, h, *[AR.conj()]*m]
    Hr = ncon(tensors, edges)
    # To be used in Ax = b solver
    b = Hr.reshape(D**2)

    l = (C.conj().T @ C).reshape(-1)
    r = np.eye(D).reshape(-1)
    Et = v.Etilde(AR, l, r)

    A_ = Et # So that Ax = b
    mvec = lambda v: A_ @ v
    A = LinearOperator((D**2, D**2), matvec=mvec)

    Rh, ecode = v.gmres(A, b, tol=tol)
    
    if ecode != 0:
        print('Sum Left gmres ecode: ', ecode)
        #raise Exception("Sum Right did not converge")
    
    return Rh

In [62]:
def update_C(hTilde, Al, Ac, Ar, C, Lh, Rh, tol=1e-5):
    tol = max(tol, 1e-14)
    D = Al.shape[0]

    def CMapOp(C_mat):
        C_map =  v.construct_CMap(Al, Ar, hTilde, Lh, Rh)
        return (C_map @ C_mat.reshape(-1)).flatten()

    COp = LinearOperator((D**2, D**2), matvec=CMapOp)
    vals, vecs = eigs(COp, k=4, which='SR', v0=C.flatten(), ncv=None, maxiter=None, tol=tol)
    print('C Map eigenvals')
    print(vals)
    C_prime = eigs(COp, k=1, which='SR', v0=C.flatten(),
                   ncv=None, maxiter=None, tol=tol)[1]

    return C_prime.reshape(D, D)

def update_Ac(hTilde, Al, Ac, Ar, C, Lh, Rh, tol=1e-5):
    tol = max(tol, 1e-14)
    D, d, _ = Al.shape

    def AcMapOp(AC):
        AC_map = v.construct_AcMap(Al, Ar, hTilde, Lh, Rh)

        return AC_map @ AC.reshape(-1)

    AC_Op = LinearOperator((d * D**2, d * D**2), matvec=AcMapOp)
    vals, vecs = eigs(AC_Op, k=4, which='SR', v0=Ac.flatten(), ncv=None, maxiter=None, tol=tol)
    print('Ac Map eigenvals')
    print(vals)
    AC_prime = (eigs(AC_Op, k=1, which='SR', v0=Ac.flatten(),
            ncv=None, maxiter=None, tol=tol)[1]).reshape(D, d, D)
    return AC_prime

In [63]:
def errorL(hTilde, Al, Ac, Ar, C, Lh, Rh):
    """
    Calculate ϵL to check for convergence.
    """
    D, d, _ = Al.shape
    AcTilde = v.construct_AcMap(Al, Ar, hTilde, Lh, Rh) @ Ac.reshape(-1)
    AcTilde = AcTilde.reshape(D, d, D)

    CTilde = v.construct_CMap(Al, Ar, hTilde, Lh, Rh) @ C.reshape(-1)
    CTilde = CTilde.reshape(D, D)

    AlCTilde = ncon((Al, CTilde), ((-1, -2, 1), (1, -3)))
    
    G = AcTilde - AlCTilde

    return np.linalg.norm(AcTilde - AlCTilde)

In [64]:
def vumps(h, D, d, A0=None, tol=1e-5, tolFactor=1e-2, maxiter=100, verbose=False, callback=None, message=False, M_opt=None):
    '''
    Perform vumps to optimise local hamiltonian h.
    '''

    if A0 is None:
        A0 = v.createMPS(D, d)
        A0 = v.normalizeMPS(A0)

    Al, Ac, Ar, C = v.mixedCanonical(A0)

    delta = tol*1e-2
    count = 0

    message_string = 'Maximum iteration reached'

    errors = np.zeros(maxiter)
    errorsL = np.zeros(maxiter)
    errorsR = np.zeros(maxiter)

    while maxiter > count:
        count += 1
        if verbose:
            print(f'iteration: {count}')

        # Rescale Hamiltonian
        hTilde = v.rescaledHnMixed(h, Ac, Ar)
        
        vals, vecs = np.linalg.eig(hTilde.reshape(4, 4))
        print('hTilde eigen vals')
        print(np.sort(vals))

        # Calculate the environments
        Lh = v.sumLeft(Al, C,  hTilde, tol=tolFactor*delta).reshape(D, D)
        Rh = v.sumRight(Ar, C, hTilde, tol=tolFactor*delta).reshape(D, D)

        # Update Ac
        #print('Updating Ac')
        AcTilde = update_Ac(hTilde, Al, Ac, Ar, C, Lh, Rh, tol=tolFactor*delta)
        #print('Change in AC')
        #print(np.sum(np.abs(AcTilde - Ac)))
        # Update C
        #print('Updating C')
        CTilde = update_C(hTilde, Al, Ac, Ar, C, Lh, Rh, tol=tolFactor*delta)
        #print('Change in C')
        #print(np.sum(np.abs(CTilde - C)))

        Al, Ac_, Ar, C_ = v.minAcCPolar2(AcTilde, CTilde, tol=delta*tolFactor**2)
        #print('Change in AC Polar')
        #print(np.sum(np.abs(Ac_ - Ac)))
        #print('Change in C Polar')
        #print(np.sum(np.abs(C_ - C)))
        Ac = Ac_
        C = C_

        # Calculate errorL
        # e = v.expValNMixed(hTilde, Ac, Ar)
        Lh = sumLeft(Al, C,  hTilde, tol=tolFactor*delta).reshape(D, D)
        Rh = sumRight(Ar, C, hTilde, tol=tolFactor*delta).reshape(D, D)
        delta = errorL(hTilde, Al, Ac, Ar, C, Lh, Rh)
        
        eL = np.linalg.norm(Ac - ncon([Al, C], ((-1, -2, 1), (1, -3)) ))
        eR = np.linalg.norm(Ac - ncon([C, Ar], ((-1, 1), (1, -2, -3)) ))
        E = np.real(v.expValNMixed(h, Ac, Ar))
        
        errors[count-1] = delta
        errorsL[count-1] = eL
        errorsR[count-1] = eR

        if verbose:
            # print(f'   energy: {E}')
            print(f'    delta: {delta}')
            print(f'    errorL: {eL}')
            print(f'    errorR: {eR}')
            print()
            # print(f'   tr_dist: {dist}')

        if callback is not None:
            callback(count, [Al, Ac, Ar, C, hTilde, Lh, Rh], delta, E)
        if delta < tol:
            message_string = 'Success'
            break

    #plt.figure()
    #plt.title('Errors L')
    #plt.plot(errors[:count], label='delta')
    #plt.plot(errorsL[:count], label='eL')
    #plt.plot(errorsR[:count], label='eR')
    #plt.legend()
    #plt.show()

    if message:
        return Al, Ac, Ar, C, message_string

    return Al, Ac, Ar, C

In [66]:
Al, Ac, Ar, C = vumps(h, 4, 2, verbose=True, tol=1e-12, maxiter=50)

iteration: 1
hTilde eigen vals
[-0.80362929+0.j -0.78382539+0.j  1.21617461+0.j  1.23597852+0.j]
Ac Map eigenvals
[-3.16557986-1.95306270e-17j -2.61986379+3.56096251e-16j
 -1.63771505+3.08811440e-16j -1.41001548+4.15193181e-16j]
C Map eigenvals
[-2.37319682+1.75893618e-16j -1.82563524-1.93387506e-16j
 -0.84301115-7.00914454e-17j -0.48471829-8.70519270e-17j]
Sum Left gmres ecode:  160
   error: 0.7954160555634647
   errorL: 0.020332424952484423
   errorR: 0.02483848335614247

iteration: 2
hTilde eigen vals
[-0.00997977+0.j  0.00982413+0.j  2.00982413+0.j  2.02962803+0.j]
Ac Map eigenvals
[-0.99507263+3.61924223e-17j  2.2436853 -4.53559655e-16j
  2.36229462-1.24908146e-16j  2.39055365+5.01606656e-17j]
C Map eigenvals
[-0.99487151-3.27675510e-16j  2.25637868+7.08680363e-16j
  2.4238244 +5.03578586e-16j  2.46074048+4.64270710e-16j]
   error: 0.00020507462282768672
   errorL: 8.307617172078296e-07
   errorR: 1.778652740534951e-05

iteration: 3
hTilde eigen vals
[-0.00977849+0.j  0.01002542+

Ac Map eigenvals
[-1.15522025-1.19227017e-16j  2.21696801-2.67625781e-16j
  2.36937829-1.06519331e-16j  2.47076599-3.65726443e-17j]
C Map eigenvals
[-1.15522025+6.87563226e-17j  2.26619673+3.05850400e-16j
  2.45687943+1.17774953e-16j  2.58688221+1.49137590e-16j]
   error: 8.775590304744711e-12
   errorL: 3.460085426268654e-14
   errorR: 7.160734766899381e-14

iteration: 22
hTilde eigen vals
[-0.00977865+0.j  0.01002525+0.j  2.01002525+0.j  2.02982916+0.j]
Ac Map eigenvals
[-1.15521938+8.60687441e-19j  2.21696863-1.38368458e-16j
  2.36937757+8.80620047e-18j  2.47076558-1.79100433e-16j]
C Map eigenvals
[-1.15521938+1.65603108e-17j  2.2661969 +1.14172083e-16j
  2.45687687+2.69220763e-16j  2.58688112-3.59319967e-17j]
   error: 4.338354569949971e-12
   errorL: 3.5270290978249005e-14
   errorR: 1.697783577701258e-14

iteration: 23
hTilde eigen vals
[-0.00977865+0.j  0.01002525+0.j  2.01002525+0.j  2.02982916+0.j]
Ac Map eigenvals
[-1.15521956+3.21761837e-16j  2.21696834-2.63641009e-16j
  2.3

In [67]:
Al, Ac, Ar, C = vumps(rho, 4, 2, verbose=True, maxiter=15)

iteration: 1
hTilde eigen vals
[-0.08432944-2.21670948e-16j  0.879473  -1.05440757e-17j
  0.88051374-6.93889390e-18j  0.88432602+1.36398661e-17j]
Ac Map eigenvals
[-0.35335533-3.15380131e-17j  1.52748918-6.02804042e-17j
  1.55036154-4.74302394e-18j  1.56501281+4.12220625e-16j]
C Map eigenvals
[-0.27162624-2.82352519e-16j  1.60786248+1.84428316e-16j
  1.63021106-1.44293903e-17j  1.71004445+1.75831051e-16j]
   error: 0.08173319709766444
   errorL: 0.00018202472641320889
   errorR: 0.0001392374793583912

iteration: 2
hTilde eigen vals
[-0.01100771-2.09114411e-16j  0.95279473-9.56087614e-18j
  0.95383547-2.42861287e-17j  0.95764775+1.83485257e-18j]
Ac Map eigenvals
[-0.18055378-4.27924778e-17j  1.7392772 +6.46481211e-17j
  1.74128535+1.49034785e-16j  1.74724866-4.39482605e-16j]
C Map eigenvals
[-0.17214356-1.46985991e-16j  1.75571868-5.82402574e-16j
  1.74874789-3.17855432e-17j  1.75039834-7.76456452e-17j]
   error: 0.008410227570667795
   errorL: 5.986715256016863e-08
   errorR: 5.7841934

In [54]:
rho_new = os.mixed_state_to_two_site_rho(Al, C, Ar)

In [58]:
print(trace_dist(rho0.reshape(4, 4), rho_new.reshape(4, 4)))
print(trace_dist(rho0.reshape(4, 4), rho0_mixed.reshape(4, 4)))
print(trace_dist(rho0_mixed.reshape(4, 4), rho_new.reshape(4, 4)))

0.010912942777431815
0.008992245538016981


In [ ]:
def get_lr(A):
    TM = ncon([A, A.conj()], ((-1, 1, -3), (-2, 1, -4))).reshape(D**2, D**2)

    vals, vecs = np.linalg.eig(TM)
    r = vecs[:, 0]
    vals, vecs = np.linalg.eig(TM.T)
    l = vecs[:, 0]
    return l, r

In [ ]:
def Etilde(A, l, r):
    D = A.shape[0]

    # Create fixed point
    fixed = ncon([l, r], ((-2,),(-1,)))

    # Create transfer matrix
    transfer = ncon([A, A.conj()], ((-1, 1, -3), (-2, 1, -4))).reshape(D**2, D**2)

    # Put together with identity
    Etilde = np.eye(D**2) - transfer + fixed

    return Etilde

In [ ]:
def get_Lh(A, l, h):
    pass

def get_Rh(A, r, h):
    pass

In [ ]:
def gradient_descent(h, D, d, maxiter=100, tol=1e-6):
    A0 = v.createMPS(D, d)
    A0 = v.normalizeMPS(A0)
    A = np.copy(A0)

    l, r = get_lr(A)
    
    e = ncon((l, r, A, A, h, A.conj(), A.conj()),
             ((1, 6), (5, 10), (1, 2, 3), (3, 4, 5), (2, 4, 7, 9), (6, 7, 8), (8, 9, 10)))
    
    h_ = h - e * np.eye(4).reshape(2, 2, 2, 2)